### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{'_wandb': {'runtime': 1}},"{'exp_name': 'hicat_traj', 'overrides': ['trai...",2t61v5l2
1,"{'epoch': 4, 't_rew_loss': 0.4903242216408253,...","{'env': {'game': 'BattleZone', 'seed': 42, 'ty...",2x0un7v1
2,"{'t_obs_loss': 1.9797599004507065, 'min_grad_n...","{'env': {'game': 'Breakout', 'seed': 42, 'type...",od0upfkk
3,"{'_step': 116000, 'epoch': 4, '_runtime': 2746...","{'env': {'game': 'Boxing', 'seed': 42, 'type':...",2gc1mgpf
4,"{'epoch': 4, 'act_f1': 0.17930685124118853, '_...","{'env': {'game': 'BankHeist', 'seed': 42, 'typ...",30r2wdhc
...,...,...,...
1202,"{'_wandb': {'runtime': 20220}, '_runtime': 201...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1203,"{'negative_sim': 0.5317903865873813, 'pos_neg_...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1204,"{'negative_sim': 0.18186782717704772, 'pos_neg...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1205,"{'_step': 117001, '_wandb': {'runtime': 17734}...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'hicat_traj'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_935/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
11,"{'min_grad_norm': 0, '_runtime': 58313, '_time...","{'env': {'game': 'UpNDown', 'seed': 42, 'type'...",2uujnio1,UpNDown
12,"{'s_obs_loss': 1.9566294714041197, 't_rew_loss...","{'env': {'game': 'Seaquest', 'seed': 42, 'type...",2hjnwflm,Seaquest
13,"{'_wandb': {'runtime': 55515}, 'act_f1': 0.151...","{'env': {'game': 'Krull', 'seed': 42, 'type': ...",1by7bgqa,Krull
14,"{'_wandb': {'runtime': 56132}, 't_rtg_loss': 0...","{'env': {'game': 'Qbert', 'seed': 42, 'type': ...",19yclb0c,Qbert
15,"{'t_rtg_loss': 0.0013160850383596228, 'min_gra...","{'env': {'game': 'KungFuMaster', 'seed': 42, '...",2xcy6t4z,KungFuMaster
16,"{'reward_f1': 0.8130985480383071, 't_act_acc':...","{'env': {'game': 'MsPacman', 'seed': 42, 'type...",1gy23ocb,MsPacman
17,"{'epoch': 10, 't_act_acc': 0.7235327889602682,...","{'env': {'game': 'PrivateEye', 'seed': 42, 'ty...",3nybtahl,PrivateEye
18,"{'max_grad_norm': 0.8215127430634124, 'min_gra...","{'env': {'game': 'Kangaroo', 'seed': 42, 'type...",2g2eeygm,Kangaroo
19,"{'_timestamp': 1670024957, 'd_act_loss': 3.425...","{'env': {'game': 'RoadRunner', 'seed': 42, 'ty...",z6638l2n,RoadRunner
20,"{'reward_f1': 0.9586374695863747, 't_act_acc':...","{'env': {'game': 'Pong', 'seed': 42, 'type': '...",2edr9t69,Pong


### 1. Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

UpNDown 2uujnio1
Seaquest 2hjnwflm
Krull 1by7bgqa
Qbert 19yclb0c
KungFuMaster 2xcy6t4z
MsPacman 1gy23ocb
PrivateEye 3nybtahl
Kangaroo 2g2eeygm
RoadRunner z6638l2n
Pong 2edr9t69
Gopher 1vyr5suk
Jamesbond 30k094ae
ChopperCommand 1iefozzs
Hero 1k59hqzs
DemonAttack 15fd2zdp
CrazyClimber 1d4eoduy
Frostbite 2jrg84k6
Freeway 3a410qaj
BattleZone 3s0chds6
Amidar 1fz4e1u1
BankHeist 2yme0xoz
Boxing 14lmyuxh
Breakout 1l1dfehl
Alien 274edgz9
Assault 3gfvb4kf
Asterix nbdr7hf3


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)